# Use SX5E and bloomberg download data to get me started

In [46]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
#initial cleaning function
def clean_sheet(dataframe):
    adj=dataframe.dropna(axis=1,how='all')
    adj=dataframe.set_index(adj.columns[0]) #use stock tickers as the index
    return adj

In [48]:
#pull the data from excel, form a dictionary of tab names and dataframes
tabs_to_grab=['Quarter','Date','Reported EPS','Comp EPS','Consensus EPS','50d MA','3d return','3d rel return','Volume','Momentum',
             'Revision','Market cap','PE','Property','Price']

data_dict={}

for s in tabs_to_grab:
    data_dict[s]=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result data\2016-2018.xlsx',sheet_name=s))

In [49]:
# grab the price time series and transform it into the same datetime format; also generate abs_return, rel_return dataframe
price=data_dict["Price"].dropna(axis=1)
new_column=[i.strftime("%d/%b/%Y") for i in price.columns.tolist() if type(i)!=str]
price.columns=new_column


## Signal 2 - based on last quarter's standing

In [127]:
## grab the date 
date=data_dict["Date"].dropna(axis=1,how='all')
adj_date=date.fillna(pd.Timestamp('1900-1-1'))

In [193]:
## form the signal 2: separate date range into a unit of 3 months, evaluate the the top and bottom quartile values, use the
# dataframe for the next quarter to generate signals, and put them into long and short buckets
start_date=pd.Timestamp(2015,12,31)
end_date=pd.Timestamp(2018,3,31)
per='Q'


def signal2_build(date_df,return_df,start_date,end_date,top_quartile,bottom_quartile):
    dt_rng=pd.date_range(start_date,end_date,freq=per)
    long_list=[]
    short_list=[]
    for s in range(len(dt_rng)-2):
        rank_list=[]
        rank=return_df[(date_df>=dt_rng[s])&(date_df<=dt_rng[s+1])]
        signal=date_df[(date_df>=dt_rng[s+1])&(date_df<=dt_rng[s+2])]
        signal_value=return_df[(date_df>=dt_rng[s+1])&(date_df<=dt_rng[s+2])]
        for index,row in rank.iterrows():
            adj_row=row.dropna()
            for val in adj_row.values:
                rank_list.append(val)
        rank_series=pd.Series(rank_list)
        top=signal[signal_value>=np.percentile(rank_series,top_quartile)].dropna(axis=1,how='all').dropna(axis=0,how='all')
        bottom=signal[signal_value<=np.percentile(rank_series,bottom_quartile)].dropna(axis=1,how='all').dropna(axis=0,how='all')
        long_list.append(top)
        short_list.append(bottom)
    long_signal=pd.concat(long_list,axis=1)
    short_signal=pd.concat(short_list,axis=1)
    return long_signal, short_signal
                
long_signal,short_signal=signal2_build(adj_date,data_dict["3d rel return"],start_date,end_date,80,20)   


In [197]:
#Stack signals into a list of tuples (stocks, dates), convert dates into a common format of datetime
def signal2_stacking(signal):
    signal2_list=[]
    for index, row in signal.iterrows():
        adj_row=row.dropna()
        for s in adj_row.values:
            ts=(s - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's') 
            adj_time=datetime.utcfromtimestamp(ts).strftime("%d/%b/%Y") #transform the datetime into a common format
            signal2_list.append((adj_row.name,adj_time))
    return signal2_list

signal2_long_list=signal2_stacking(long_signal)
signal2_short_list=signal2_stacking(short_signal)

C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  import sys


In [194]:
# grab the price time series and transform it into the same datetime format; also generate abs_return, rel_return dataframe
price=data_dict["Price"].dropna(axis=1)
new_column=[i.strftime("%d/%b/%Y") for i in price.columns.tolist() if type(i)!=str]
price.columns=new_column

abs_return=price.diff(1,axis=1)/price
abs_return=abs_return.dropna(how='all',axis=1)
rel_return=abs_return - abs_return.iloc[-1]
rel_return=rel_return.dropna(how='all',axis=1)